In [21]:
import pandas as pd

In [22]:
# read csv data
df = pd.read_csv('results/top_contribuyentes.csv')

In [23]:
# 0. Drop Unnamed column and index DF column
df = df.drop('Unnamed: 0', axis=1)

#### More cleaning

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   RUC                    4000 non-null   float64
 1   NOMBRE O RAZON SOCIAL  4500 non-null   object 
 2   APORTE TOTAL(mm)       4500 non-null   int64  
 3   ANHO                   4500 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 140.8+ KB


In [25]:
# replace NaN values on column RUC with 0
df['RUC'] = df['RUC'].fillna(0)
# parse RUC column to int and then to string
df['RUC'] = df['RUC'].astype(int).astype(str)
# parse ANHO column to int
df['ANHO'] = df['ANHO'].astype(int)

In [26]:
# create new DF excluding data form 2020
df_no_2020 = df[df['ANHO'] != 2020]
# create a new DF with only RUC and NOMBRE O RAZON SOCIAL that finds unique values of RUC and NOMBRE O RAZON SOCIAL
df_nombre_ruc = df_no_2020[['RUC', 'NOMBRE O RAZON SOCIAL']].drop_duplicates(subset=['RUC', 'NOMBRE O RAZON SOCIAL'], keep='first')
df_nombre_ruc

,RUC,NOMBRE O RAZON SOCIAL
0,80009735,ADMINISTRACION NACIONAL DE ELECTRICIDAD - ANDE
1,80008790,TABACALERA DEL ESTE SA
2,80002201,BANCO ITAU PARAGUAY S.A
3,80003400,PARAGUAY REFRESCOS SA
4,80086846,CERVEPAR S.A.
...,...,...
4481,80045670,GRUPO SEGURIDAD Y VIGILANCIA PARAGUAYA SOCIED...
4490,80028982,COMVENCE SA
4493,80029571,AGRO-ECO SRL
4497,80002449,SUPERMERCADO GUARANI SRL


In [27]:
# Join and replace RUC column based on NOMBRE O RAZON SOCIAL from df with df_nombre_ruc for 2020
df_2020 = df[df['ANHO'] == 2020]
df_2020 = df_2020.merge(df_nombre_ruc, on='NOMBRE O RAZON SOCIAL', how='left')
df_2020 = df_2020.drop('RUC_x', axis=1)
df_2020 = df_2020.rename(columns={'RUC_y': 'RUC'})
df_2020.head()

,NOMBRE O RAZON SOCIAL,APORTE TOTAL(mm),ANHO,RUC
0,ADMINISTRACION NACIONAL DE ELECTRICIDAD - ANDE,407807,2020,80009735
1,TABACALERA DEL ESTE SA,242106,2020,80008790
2,BANCO ITAU PARAGUAY S.A,201577,2020,80002201
3,BANCO NACIONAL DE FOMENTO,157473,2020,80000856
4,CERVEPAR S.A.,154792,2020,80086846


In [28]:
# Drop lines from 2020 with RUC == 0 and concat data from df_2020 with df
df = df[df['ANHO'] != 2020]
df = pd.concat([df, df_2020], axis=0)
df

,RUC,NOMBRE O RAZON SOCIAL,APORTE TOTAL(mm),ANHO
0,80009735,ADMINISTRACION NACIONAL DE ELECTRICIDAD - ANDE,336481,2021
1,80008790,TABACALERA DEL ESTE SA,198331,2021
2,80002201,BANCO ITAU PARAGUAY S.A,174807,2021
3,80003400,PARAGUAY REFRESCOS SA,147677,2021
4,80086846,CERVEPAR S.A.,146989,2021
...,...,...,...,...
495,80013889,DATA SYSTEMS SA EMISORA DE CAPITAL ABIERTO,2669,2020
496,80022458,PROQUITEC SA,2662,2020
497,80008762,GAS METAL SRL,2657,2020
498,80001513,NUEVA AMERICANA SA,2649,2020


In [29]:
# order DF by APORTE TOTAL(mm) per every year
df = df.sort_values(by=['ANHO', 'APORTE TOTAL(mm)'], ascending=False)
# add a new column with the position of the contributor per year
df['POSICION'] = df.groupby('ANHO').cumcount() + 1
df.head(20)

,RUC,NOMBRE O RAZON SOCIAL,APORTE TOTAL(mm),ANHO,POSICION
0,80009735,ADMINISTRACION NACIONAL DE ELECTRICIDAD - ANDE,336481,2021,1
1,80008790,TABACALERA DEL ESTE SA,198331,2021,2
2,80002201,BANCO ITAU PARAGUAY S.A,174807,2021,3
3,80003400,PARAGUAY REFRESCOS SA,147677,2021,4
4,80086846,CERVEPAR S.A.,146989,2021,5
5,80019270,BANCO CONTINENTAL SA EMISORA DE CAPITAL ABIERTO,142276,2021,6
6,80000856,BANCO NACIONAL DE FOMENTO,139376,2021,7
7,80009310,VISION BANCO S.A.E.C.A,93627,2021,8
8,80034461,SUDAMERIS BANK SAECA,92594,2021,9
9,80003457,CERVECERIA PARAGUAYA SA,85477,2021,10


Lineas que no tienen RUC y solo aparecen en 2020

In [30]:
# Drop lines with NaN values on RUC column
df = df.dropna(subset=['RUC'])

# Encontrar el top 10 por cada anho

#### Ya que esta ordenado y con su posicion, solamente buscamos los que quedaron en el top 10 por cada anho

In [31]:
# Find the top 10 contributors per year
df_top10 = df[df['POSICION'] <= 10]
df_top10.head()

,RUC,NOMBRE O RAZON SOCIAL,APORTE TOTAL(mm),ANHO,POSICION
0,80009735,ADMINISTRACION NACIONAL DE ELECTRICIDAD - ANDE,336481,2021,1
1,80008790,TABACALERA DEL ESTE SA,198331,2021,2
2,80002201,BANCO ITAU PARAGUAY S.A,174807,2021,3
3,80003400,PARAGUAY REFRESCOS SA,147677,2021,4
4,80086846,CERVEPAR S.A.,146989,2021,5


In [32]:
df_top10.tail()

,RUC,NOMBRE O RAZON SOCIAL,APORTE TOTAL(mm),ANHO,POSICION
4005,80021825,BRASFUMO DEL PARAGUAY SA,79907,2013,6
4006,80003400,PARAGUAY REFRESCOS SA,78491,2013,7
4007,80019270,BANCO CONTINENTAL SA EMISORA DE CAPITAL ABIERTO,75595,2013,8
4008,80016096,RETAIL S.A.,60929,2013,9
4009,80020981,BANCO REGIONAL S.A.E.C.A.,50728,2013,10


# Encontrar cuantos son Personas Fisicas y cuantos son Personas Juridicas

### Buscar en base al Nro de RUC

In [33]:
# Crear una columna con Fisica o Juridica dependiendo de si el RUC empieza o no con un 8
df['TIPO'] = df['RUC'].apply(lambda x: 'Juridica' if x[0] == '8' else 'Fisica')

In [34]:
# Imprimir cuantos contribuyentes fisicos y juridicos hay
print('Contribuyentes fisicos: ', df[df['TIPO'] == 'Fisica'].shape[0])
print('Contribuyentes juridicos: ', df[df['TIPO'] == 'Juridica'].shape[0])

Contribuyentes fisicos:  86
Contribuyentes juridicos:  4359


# Hallar cuantos contribuyes estan en todos los anhos

In [35]:
# Cuantos anhos distintos hay
print('Anhos distintos: ', df['ANHO'].unique())

Anhos distintos:  [2021 2020 2019 2018 2017 2016 2015 2014 2013]


In [38]:
# Cuantos RUCs y NOMBRE O RAZON SOCIAL estan presentes en todos los años
df_ruc = df[['RUC', 'ANHO', 'NOMBRE O RAZON SOCIAL']].drop_duplicates(subset=['RUC', 'ANHO', 'NOMBRE O RAZON SOCIAL'], keep='first')
df_ruc = df_ruc.groupby(['RUC', 'NOMBRE O RAZON SOCIAL']).count()
# segun la cantidad de anhos, buscar los RUCs que se repitan la msima cantidad de veces
df_ruc = df_ruc[df_ruc['ANHO'] == len(df['ANHO'].unique())]
print('RUCs presentes en todos los años: ', df_ruc.shape[0])
df_ruc

RUCs presentes en todos los años:  139


,,ANHO
RUC,NOMBRE O RAZON SOCIAL,
80000089,INDUSTRIA NACIONAL DEL CEMENTO,9
80000423,DIRECC.NACIONAL DE AERONAUTICA CIVIL D.I.N.A.C.,9
80000856,BANCO NACIONAL DE FOMENTO,9
80001129,AUTOMOVIL SUPPLY SA,9
80001340,AUTOMOTOR SA,9
...,...,...
80044383,MATRISOJA S.A.,9
80050476,MOBILE CASH PARAGUAY SA,9
80050807,A V SOCIEDAD ANONIMA,9


In [39]:
df_ruc_cant_anhos = df[['RUC', 'ANHO', 'NOMBRE O RAZON SOCIAL']].drop_duplicates(subset=['RUC', 'ANHO', 'NOMBRE O RAZON SOCIAL'], keep='first')
df_ruc_cant_anhos = df_ruc_cant_anhos.groupby(['RUC', 'NOMBRE O RAZON SOCIAL']).count()
df_ruc_cant_anhos

,,ANHO
RUC,NOMBRE O RAZON SOCIAL,
1056702,HUTZ HUGO GERMAN,6
1242414,PRATAS DA COSTA JULIO,1
1268650,SPERANZA JARA LEOPOLDO FABIO,1
1547085,FERNANDEZ DE ALCARAZ PATRICIA ESTHER,3
1556191,SIMON JOSEMIR TADEU,2
...,...,...
918463,FERREIRA LOPEZ NATHALIA MARIA,1
939309,DORIA CORTES RAMON ALBERTO,1
941152,HUTZ GERMAN,7


In [42]:
# crear un DF con cada NOMBRE O RAZON SOCIAL unicos y sus ANHOS en formato de lista
df_ruc_cant_anhos = df.reset_index()
df_ruc_cant_anhos = df_ruc_cant_anhos[['RUC', 'NOMBRE O RAZON SOCIAL', 'ANHO']]
df_ruc_cant_anhos = df_ruc_cant_anhos.groupby(['RUC', 'NOMBRE O RAZON SOCIAL']).agg({'ANHO': lambda x: list(x)})
df_ruc_cant_anhos

,,ANHO
RUC,NOMBRE O RAZON SOCIAL,
1056702,HUTZ HUGO GERMAN,"[2021, 2019, 2017, 2016, 2015, 2014]"
1242414,PRATAS DA COSTA JULIO,[2014]
1268650,SPERANZA JARA LEOPOLDO FABIO,[2018]
1547085,FERNANDEZ DE ALCARAZ PATRICIA ESTHER,"[2019, 2015, 2014]"
1556191,SIMON JOSEMIR TADEU,"[2019, 2013]"
...,...,...
918463,FERREIRA LOPEZ NATHALIA MARIA,[2013]
939309,DORIA CORTES RAMON ALBERTO,[2021]
941152,HUTZ GERMAN,"[2021, 2019, 2018, 2016, 2015, 2014, 2013]"


## Estadistica

In [48]:
# Hallar el promedio de APORTE TOTAL(mm) por ANHO
df_promedio = df.groupby(['ANHO']).mean()
df_promedio = df_promedio.reset_index()
df_promedio = df_promedio[['ANHO', 'APORTE TOTAL(mm)']]
df_promedio = df_promedio.rename(columns={'APORTE TOTAL(mm)': 'PROMEDIO APORTE TOTAL(mm)'})
df_promedio

/tmp/ipykernel_3886/263181746.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_promedio = df.groupby(['ANHO']).mean()


,ANHO,PROMEDIO APORTE TOTAL(mm)
0,2013,8644.404000
1,2014,9692.652000
2,2015,10417.446000
3,2016,12153.212000
4,2017,12311.934000
5,2018,13024.780000
6,2019,14095.200000
7,2020,13204.786517
8,2021,13200.248000
